In [ ]:
%load_ext dotenv
%dotenv

import os

In [ ]:
%reload_ext dotenv
%dotenv

import os

http://localhost:9200/_cat/indices?v

In [ ]:
from os import listdir
from os.path import isfile, join

import re

mypath = "."
onlyjsonfiles = [ re.search('central-[\d]{4}\.[\d]{2}\.[\d]{2}', f).group(0) 
                 for f in listdir(mypath) 
                 if isfile(join(mypath, f)) 
                 and f.split(".")[-1] == 'json' 
                 and re.search('central-[\d]{4}\.[\d]{2}\.[\d]{2}-documents', f) != None  
                ]

onlyjsonfiles

In [ ]:
import requests

In [ ]:
import re

res = requests.get(os.environ["ELASTIC_HOST"] + "/_cat/indices?format=json")
list_indices =  [ str(index['index']) 
                 for index in res.json() 
                 if re.search('central-[\d]{4}\.[\d]{2}\.[\d]{2}', index['index']) ]

list_indices

In [ ]:
indices_to_backup = [item for item in list_indices if item not in onlyjsonfiles]
indices_to_backup

In [ ]:
for index in indices_to_backup:
    !elasticbackup localhost:9200 $index

In [ ]:
from minio import Minio
from minio.error import ResponseError

In [ ]:
minioClient = Minio(os.environ["S3_HOST"],
                  access_key=os.environ["ACCESS_KEY"],
                  secret_key=os.environ["SECRET_KEY"],
                  secure=True)

In [ ]:
bucket_elastic = "elastic"

In [ ]:
found_bucket_elastic = False
for bucket in minioClient.list_buckets():
    if bucket.name == bucket_elastic:
        found_bucket_elastic = True
        break
    #print(bucket.name, bucket.creation_date)

In [ ]:
if minioClient.bucket_exists(bucket_elastic) == False:
    minioClient.make_bucket("elastic", location="us-east-1")

In [ ]:
onlyjsonfiles = [ f 
                 for f in listdir(mypath) 
                 if isfile(join(mypath, f)) 
                 and f.split(".")[-1] == 'json' 
                 #and re.search('central-[\d]{4}\.[\d]{2}\.[\d]{2}-documents', f) != None  
                ]

onlyjsonfiles

In [ ]:
already_backuped = [ obj.object_name for obj in minioClient.list_objects(bucket_elastic) ]
already_backuped

In [ ]:
to_backup = [item for item in onlyjsonfiles if item not in already_backuped]
to_backup

In [ ]:
for file in to_backup:
    minioClient.fput_object(bucket_elastic, file, './' + file)